# Consiguiendo los Data Frames

In [1]:
# Import modules
import coinmetrics
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pprint
import concurrent.futures
import time

# Initialize a reference object, in this case "cm" for the Community API
cm = coinmetrics.Community()

In [2]:
# Consigo todos los assets con datos
supported_assets = cm.get_supported_assets()

In [3]:
# Elijo la ventana de observaciones
END_DATE = datetime.now()  
INIT_DATE = END_DATE - relativedelta(days=1) 

INIT_STR = INIT_DATE.strftime("%Y-%m-%d")
END_STR = END_DATE.strftime("%Y-%m-%d")

asset_data = dict()

# Agarro la data que me interesa

lista = ['AdrActCnt', 'BlkCnt', 'CapMVRVCur', 'CapMVRVFF', 'CapMrktFFUSD',
         'CapRealUSD', 'NVTAdjFF', 'NVTAdjFF90', 'FeeMeanUSD', 'FeeMedUSD',
         'IssContPctAnn', 'IssContPctAnn', 'NVTAdj90', 'RevUSD', 'RevAllTimeUSD',
         'TxCntSec', 'TxTfrValMeanUSD', 'VelCur1yr']

In [4]:
# Defino las funciones que imprimen la descripción de las variables y que consiguen el dataframe de cada asset.

def metric_info(metric):
    i = cm.get_metric_info(metric)
    print('Name: ' + i[0]['name'] + '\n' + 
    'Category: ' + i[0]['category'] + '\n' + 
    'Subcategory: ' + i[0]['subcategory'] + '\n' + 
    'id: ' + i[0]['id'] + '\n' + 
    'Description: ' + i[0]['description'] + '\n\n')

def get_df(asset):
   
    metricas = cm.get_available_data_types_for_asset(asset)
    
    # hay que encontrar la interseccion entre la data disponible y la que me interesa. Sino, sale error.
    intersection = set(metricas).intersection(lista)  

    string = ""
    for item in intersection:
        string = string + item + ','
    data = cm.get_asset_data_for_time_range(asset, string[:-1], INIT_STR, END_STR)
    
    df = coinmetrics.cm_to_pandas(data)
    asset_data[asset] = df    

In [5]:
# imprime la descripcion de los datos que consideré más importante

with concurrent.futures.ThreadPoolExecutor() as executor: 
    executor.map(metric_info, lista)    

Name: Capitalization, MVRV, free float
Category: Market
Subcategory: Market Capitalization
id: CapMVRVFF
Description: The ratio of the free float market capitalization to the sum realized USD value of the current supply.


Name: Issuance, continuous, percent, annualized
Category: Supply
Subcategory: Issuance
id: IssContPctAnn
Description: The percentage of new native units (continuous) issued over that interval, extrapolated to one year (i.e., multiplied by 365), and divided by the current supply at the end of that interval. Also referred to as the annual inflation rate.


Name: Fees, transaction, median, USD
Category: Fees and Revenue
Subcategory: Fees
id: FeeMedUSD
Description: The USD value of the median fee per transaction that interval.


Name: Issuance, continuous, percent, annualized
Category: Supply
Subcategory: Issuance
id: IssContPctAnn
Description: The percentage of new native units (continuous) issued over that interval, extrapolated to one year (i.e., multiplied by 365), a

In [6]:
start = time.perf_counter()

# consigo los dataframes y los guardo en un dict.

with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(get_df, supported_assets)    
    
finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

# Te queda un dict de dfs, donde las keys son los simbolos.

Finished in 41.24 second(s)
